In [ ]:
%pip install langchain --quiet
%pip install accelerate --quiet
%pip install transformers --quiet
%pip install bitsandbytes --quiet
%pip install unstructured --quiet
%pip install sentence-transformers --quiet
%pip install -U faiss-gpu tiktoken
%pip install langchainhub --quiet
%pip install pinecone-client --quiet
%pip install langchain_pinecone --quiet
%pip install rank_bm25 --quiet
%pip install python-dotenv --quiet

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/nlp_chatbot_folder

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/nlp_chatbot_folder


In [15]:
import yaml
with open("cfg.yaml", "r") as file:
    cfg = yaml.safe_load(file)
cfg

{'retrievers': {'bm25': {'topk': 3, 'path': 'chunked_docs.pkl'},
  'faiss': {'faiss_index_path': 'faiss_indices/',
   'search_type': 'similarity_score_threshold',
   'score_threshold': 0.77,
   'topk': 3}},
 'ensemble': {'weights': [0.4, 0.6]},
 'llm_model': {'name': 'mistralai/Mistral-7B-Instruct-v0.1',
  'local_path': '',
  'temperature': 0.001,
  'top_p': 0.95,
  'repetition_penalty': 1.15},
 'embedding_model': 'thenlper/gte-base',
 'use_qa_chain': True,
 'use_conversational_qa_chain': True,
 'use_multi_query_qa_chain': True,
 'conversational_chain': {'conversation_depth': 2}}

In [16]:
from custom_chatbot import MedicalChatbot

In [17]:
chatbot = MedicalChatbot(cfg)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
unrelated_queries = [
    "What is the best recipe for homemade pizza?",
    "How to change a car tire in 5 steps?",
    "Explain the detailed history of the Roman Empire.",
    "Why was the Eiffel Tower built?"
]

queries_in_other_languages = [
    "Wie könnten Fortschritte in der künstlichen Intelligenz zur Früherkennung und personalisierten Behandlung der Alzheimer-Krankheit beitragen?",
    "Qual è il potenziale dell'apprendimento automatico per prevedere gli esiti dei pazienti sulla base dell'analisi delle immagini mediche in oncologia?",
    "高血压的早期症状有哪些",
    "Какие существуют методы ранней диагностики рака груди?"
]

one_word_queries = [
    "genetics",
    "neuroimaging",
    "intelligence",
    "Antibiotics?"
]

very_long_queries = [
    "I've been feeling a bit under the weather lately, starting with a mild headache that gradually became more persistent over the last week. Accompanying the headache, there's been an intermittent fever that doesn't seem to go away completely, even after taking over-the-counter fever reducers. Furthermore, I've noticed a rash that started around the same time as the fever, initially just a few spots but now spreading across my chest and back. It's not particularly itchy but is quite noticeable. Additionally, my joints, especially in the hands and knees, have been feeling stiff and sometimes painfully swollen. I've been trying to drink more fluids and rest, but there hasn't been much improvement. Given these symptoms, what might be the underlying condition, and what steps should I take next?",
    "Considering the rapid advancements in artificial intelligence and its integration into biomedical research, particularly in the field of genomics where vast amounts of data are generated and analyzed, how could the development of more sophisticated AI algorithms enhance our understanding of complex genetic disorders, potentially leading to breakthroughs in identifying novel genetic markers for early diagnosis and the creation of more personalized, effective treatment strategies tailored to the individual genetic makeup of patients, thereby revolutionizing patient care in the realm of personalized medicine?",
    "Given the rapid advancements in medical technology and pharmaceuticals, particularly in the fields of genomics and personalized medicine, which promise to revolutionize the treatment of a wide range of diseases from cancer to rare genetic disorders, it's intriguing to consider the ethical, legal, and social implications of these developments, such as the accessibility of these treatments, privacy concerns around genetic information, and the potential for creating disparities in healthcare outcomes among different populations, raising the question of how society can prepare for and address these challenges.",
    "Considering the vast array of dietary advice available online and in various health publications, ranging from plant-based diets to keto and paleo, each claiming numerous health benefits from weight loss to improved energy levels and reduced risk of chronic diseases, it's increasingly difficult to discern which diet might be the most beneficial for someone with a history of cardiovascular issues and type 2 diabetes, taking into account the need for balanced nutrition and the potential impact on blood sugar levels and heart health, leading to the question of how one should navigate these dietary recommendations to make informed decisions about managing their health."
]

poorly_structured_queries = [
    "Leg pain after running remedy?",
    "Headache fever tired what do?",
    "leg pain sitting long why?",
    "good food high blood pressure?"
]

medical_questions = [
    "How do Alzheimer's disease and other forms of dementia impact intelligence?",
    "How does emotional intelligence impact leadership effectiveness and team dynamics?",
    "Can social intelligence be improved through training, and what methods are most effective?",
    "How does the Flynn effect challenge the understanding of IQ tests over time?"
]

In [25]:
from collections import OrderedDict
edge_questions = OrderedDict({
    "unrelated_queries": unrelated_queries,
    "queries_in_other_languages": queries_in_other_languages,
    "one_word_queries": one_word_queries,
    "very_long_queries": very_long_queries,
    "poorly_structured_queries": poorly_structured_queries,
    "medical_queries_(non_edge)": medical_questions
})

In [26]:
similarity_score_list = [0.74, 0.77, 0.80, 0.83, 0.86, 0.89]

In [27]:
from collections import OrderedDict
edge_question_responses = OrderedDict({key: OrderedDict({key: [] for key in edge_questions.keys()}) for key in similarity_score_list})
edge_question_responses

OrderedDict([(0.74,
              OrderedDict([('unrelated_queries', []),
                           ('queries_in_other_languages', []),
                           ('one_word_queries', []),
                           ('very_long_queries', []),
                           ('poorly_structured_queries', []),
                           ('medical_queries_(non_edge)', [])])),
             (0.77,
              OrderedDict([('unrelated_queries', []),
                           ('queries_in_other_languages', []),
                           ('one_word_queries', []),
                           ('very_long_queries', []),
                           ('poorly_structured_queries', []),
                           ('medical_queries_(non_edge)', [])])),
             (0.8,
              OrderedDict([('unrelated_queries', []),
                           ('queries_in_other_languages', []),
                           ('one_word_queries', []),
                           ('very_long_queries', []),
             

In [28]:
def output_response(response):
    print(f"query: {response['query']}")
    print(f"retrieved document count: {len(response['source_documents'])}")
    print(f"response: {response['result']}")

In [29]:
for similarity_score in similarity_score_list:
    print(f"\n*** SIMILARITY SCORE THRESHOLD: {similarity_score} ***")
    chatbot.dense_retriever.search_kwargs["score_threshold"] = similarity_score
    for question_type, question_list in edge_questions.items():
        print(f"** {question_type.replace('_', ' ').title()} **")
        for question in question_list:
            response = chatbot.generate_response(question, return_raw=True)
            output_response(response)
            print("\n")
            edge_question_responses[similarity_score][question_type].append(response)
        print("****************************************")
    print("---------------------------------------------------")



*** SIMILARITY SCORE THRESHOLD: 0.74 ***
** Unrelated Queries **
query: What is the best recipe for homemade pizza?
retrieved document count: 0
response: Sorry, but I don't know as my capabilities are focused on medical assistance


query: How to change a car tire in 5 steps?
retrieved document count: 0
response: Sorry, but I don't know as my capabilities are focused on medical assistance


query: Explain the detailed history of the Roman Empire.
retrieved document count: 0
response: Sorry, but I don't know as my capabilities are focused on medical assistance


query: Why was the Eiffel Tower built?
retrieved document count: 0
response: Sorry, but I don't know as my capabilities are focused on medical assistance


****************************************
** Queries In Other Languages **


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


query: Wie könnten Fortschritte in der künstlichen Intelligenz zur Früherkennung und personalisierten Behandlung der Alzheimer-Krankheit beitragen?
retrieved document count: 6
response:  I am sorry, but I do not see any direct connection between the provided abstracts and the question about how artificial intelligence (AI) can contribute to early detection and personalized treatment of Alzheimer's disease. Could you please provide me with more specific information or context about the question so that I can assist you better?




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


query: Qual è il potenziale dell'apprendimento automatico per prevedere gli esiti dei pazienti sulla base dell'analisi delle immagini mediche in oncologia?
retrieved document count: 6
response:  I am sorry, but I do not see any information about the potential of automatic learning for predicting the outcomes of cancer patients based on image analysis in oncology. Could you please provide me with more context or clarify your question?




...

query: How does emotional intelligence impact leadership effectiveness and team dynamics?
retrieved document count: 6
response:  Based on the given context, emotional intelligence (EI) has a significant impact on leadership effectiveness and team dynamics. Several studies have shown that EI improves both behaviors and business results and has a positive impact on work team performance. Additionally, EI is positively related to team members' attitudes about work. The new trends focus on the impacts of COVID-19, the global crisis, working in VUCEA and BANI environments, comparative studies between generations, the application of artificial intelligence, and the influence of mindfulness on organizations. Overall, EI plays a crucial role in enhancing leadership effectiveness and team dynamics in various industries and settings.


query: Can social intelligence be improved through training, and what methods are most effective?
retrieved document count: 0
response: Sorry, but I don't know as

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


query: How does the Flynn effect challenge the understanding of IQ tests over time?
retrieved document count: 6
response:  The Flynn effect challenges the understanding of IQ tests over time because it suggests that IQ scores are increasing over time, resulting in norms becoming obsolete. While the Flynn effect is widely accepted, efforts to estimate its magnitude and error of measurement have been controversial and have prevented determination of factors that moderate the Flynn effect across different IQ tests. A metaanalysis was conducted to determine the magnitude of the Flynn effect with a higher degree of precision to determine the error of measurement and to assess the impact of several moderator variables on the mean effect size. The metaanalytic mean was 231.95 standard score points per decade, and the mean effect size for 53 comparisons was 293.95 IQ points per decade. However, these results did not support the hypothesis that the Flynn effect is diminishing. The Flynn effect 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


query: How does the Flynn effect challenge the understanding of IQ tests over time?
retrieved document count: 2
response:  The Flynn effect challenges the understanding of IQ tests over time by suggesting that IQ scores may be increasing due to various factors such as education, nutrition, and other environmental influences. This increase in scores can result in norms becoming obsolete, making it difficult to accurately compare IQ scores across different generations or even within the same generation. Additionally, the Flynn effect has been found to vary depending on the type of IQ test used, which can further complicate the interpretation of IQ scores. Overall, the Flynn effect highlights the need for ongoing research and development of IQ tests to ensure their accuracy and reliability over time.


****************************************
---------------------------------------------------


In [31]:
# Generating the markdown file content
md_content = "# Chatbot Edge Questions Analysis\n\n"

# Adding an explanation in markdown format to the existing file to describe what the analysis represents
md_content += """
## Explanation of Analysis

This document provides an analysis of how a medical chatbot handles a variety of "edge case" questions. These questions fall into different categories, such as unrelated queries, queries in other languages, one-word queries, very long queries, poorly structured queries, and medical-related queries. The chatbot's responses are evaluated based on three different similarity score thresholds (0.75, 0.80, 0.85) to determine how these thresholds affect the chatbot's ability to provide relevant answers or admit lack of knowledge by responding with "I don't know".

### Key Points:

- **Summary Table:** Shows the count of questions in each category for them at least one document retrieved by the retriever so that were answered without resorting to "Sorry, but I don't know as my capabilities are focused on medical assistance", across different similarity score thresholds. This helps in understanding which similarity threshold provides the best balance between providing answers and acknowledging when it cannot provide a relevant response.

- **Detailed Responses:** Lists each question along with the chatbot's response and the number of source documents retrieved (if any), for each similarity score threshold. This detailed breakdown allows for a closer examination of how the chatbot's performance varies not just between different categories of questions, but also how it adjusts its responses based on the similarity score.

The purpose of this analysis is to fine-tune the chatbot's parameters for optimal performance, ensuring it can handle a wide range of queries effectively, while also recognizing its limitations in certain scenarios.
"""


# Adding a table header
md_content += "## Summary Table\n\n"

# Calculating the number of questions answered without "I don't know" for each category and similarity score
from collections import OrderedDict
summary_table = OrderedDict({key: OrderedDict() for key in edge_question_responses.keys()})
for similarity_score in edge_question_responses.keys():
    for category in edge_question_responses[similarity_score].keys():
        answered_without_idk = sum(1 for response in edge_question_responses[similarity_score][category] if len(response["source_documents"])>0)
        summary_table[similarity_score][category] = answered_without_idk


md_content += f"| Similarity Threshold"
for category, score in summary_table[list(summary_table.keys())[0]].items():
    md_content += f"| {category.replace('_', ' ').title()} "
md_content += "|\n|-------"
for score, category in summary_table[list(summary_table.keys())[0]].items():
    md_content += f"|-------"
md_content += "|\n"

# Adding data to the table
for score, categories in summary_table.items():
    md_content += f"| {score} "
    for category, answered_without_idk in summary_table[score].items():
        md_content += f"| {answered_without_idk} / {len(edge_questions[category])}"
    md_content += "|\n"

# Adding detailed responses
md_content += "\n## Detailed Responses\n\n"

for score in similarity_score_list:
    md_content += f"### Similarity Score: {score}\n\n"
    for category, questions in edge_questions.items():
        md_content += f"#### {category.replace('_', ' ').title()}\n"
        for i, question in enumerate(questions):
            response = edge_question_responses[score][category][i]
            md_content += f"- **Question:** {question}\n"
            md_content += f"  - **Response:** {response['result']}\n"
            md_content += f"  - **Retrieved Document Count:** {len(response['source_documents'])}\n\n"

# Writing the md_content to a file
md_file_path = "edge_questions_analysis.md"
with open(md_file_path, "w", encoding="utf-8") as md_file:
    md_file.write(md_content)

